# Week 3 project

Importing Essential Libraries

In [2]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium



    we shall be reading the following website: 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
    Then we would check for status code and headers



In [11]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=890001695'
result = requests.get(url)
print(url)
print(result.status_code)
print(result.headers)

https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=890001695
200
{'Date': 'Fri, 27 Mar 2020 18:21:11 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Server': 'mw1413.eqiad.wmnet', 'X-Powered-By': 'PHP/7.2.26-1+0~20191218.33+debian9~1.gbpb5a340+wmf1', 'X-Content-Type-Options': 'nosniff', 'P3P': 'CP="See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-language': 'en', 'Vary': 'Accept-Encoding,Cookie,Authorization', 'Expires': 'Fri, 27 Mar 2020 18:21:11 GMT', 'Last-Modified': 'Wed, 18 Mar 2020 17:12:23 GMT', 'Backend-Timing': 'D=340969 t=1585333271343119', 'X-ATS-Timestamp': '1585333271', 'Content-Encoding': 'gzip', 'X-Varnish': '328939684', 'Age': '0', 'X-Cache': 'cp5010 miss, cp5008 pass', 'X-Cache-Status': 'pass', 'Server-Timing': 'cache;desc="pass"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Set-Cookie': 'WMF-Last-Access=27-Mar-2020;Path=/;HttpOnly;secure;Expires=Tue, 28 Apr 2

Now we will proceed to create a dataframe and clean the data to gather the information that is relevant to us.

In [12]:
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])


In [13]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)
df.head()
df.shape

(211, 3)

Here we will group the dataframe by postalcodes then we will keep the first borough and join the neighbourhoods

In [14]:
df = df.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()

Now we will check out if it has worked correctly using 'M5R' as an example.

In [18]:
df[df['PostalCode']=='M5R']

,PostalCode,Borough,Neighborhood
65,M5R,Central Toronto,"The Annex, North Midtown, Yorkville"


Now we shall convert the above data into a pandas dataframe

Now we will check the shape again

In [19]:
df.shape

(103, 3)